In [1]:
import pandas as pd
import functions as fn
import json

In [2]:
file_path = "./data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/"
df = pd.read_csv(file_path+"discharge.csv")
type(df)

pandas.core.frame.DataFrame

In [5]:
### NOTE: only need to run this once - creates a file containing all the processed information ready for annotation
file_path = "./data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/"
df = pd.read_csv(file_path+"discharge.csv")
all_cards = []
for i in range(len(df)):
    processor = fn.AnnotationProcess(df["text"][i])
    cards = processor.clean_report()
    for card in cards:
        all_cards.append(card)

with open('./data/cards.json', "w") as f:
    json.dump(all_cards, f)

#with open("./data/processed_text.txt", "w") as f:
#    f.write(text)

In [3]:
text = df["text"][50]

with open("./consultation_notes.txt", "w") as f:
    f.write(text)

In [5]:
text

' \nName:  ___             Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nIV Dye, Iodine Containing Contrast Media / Oxycodone / \ncilostazol\n \nAttending: ___.\n \nChief Complaint:\nPneumonia identified by outpatient CXR\n\n \nMajor Surgical or Invasive Procedure:\nNone\n\n \nHistory of Present Illness:\n___ PMH of Asthma, COPD, tobacco use,  p/w 4 days of "cold like" \nsymptoms. Pt noted that she has a chronic cough at baseline, \nworse at night, but noted that her cough was now productive of \n___ sputum and was a/w wheezing and subjective fevers (but did \nnot take her temperature). Pt also noted that baseline DOE \nincreased to mild shortness of breath at rest during the same \ntime period. Pt took tylenol and robitussin to good effect. Pt \ndenied any CP, but noted that she felt her heart "flutter" \nduring this time period. No changed in her 2 pillow orthopnea.\n\nPt 

In [2]:
dir = "./data"
label_list = [
   "irAE_symptoms",
   "irAE_investigations"
]
database = "irAE"
model = "en_core_web_sm"
annotate = fn.ProdigyFunctions(dir, label_list, database, model)

annotate.database_connect()

annotate.prodigy_connect("ner.manual", "./data/cards.json")


Using 2 label(s): irAE_symptoms, irAE_investigations

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

